In [1]:
import tensorflow
tf = tensorflow
print(tensorflow.__version__)

2.1.0


In [2]:
with open('LOTR_Fellowship_Text.txt', 'rb') as fin:
    lotr_1_rtext_b = fin.read()
lotr_1_rtext = lotr_1_rtext_b.decode()
lotr_1_rtext[:1000]

'\r\nBook I\r\n\r\n \r\nChapter 1\r\nA Long-expected Party\r\n\r\nWhen Mr. Bilbo Baggins of Bag End announced that he would shortly be celebrating his eleventy-first birthday with a party of special magnificence, there was much talk and excitement in Hobbiton.\r\n\r\nBilbo was very rich and very peculiar, and had been the wonder of the Shire for sixty years, ever since his remarkable disappearance and unexpected return. The riches he had brought back from his travels had now become a local legend, and it was popularly believed, whatever the old folk might say, that the Hill at Bag End was full of tunnels stuffed with treasure. And if that was not enough for fame, there was also his prolonged vigour to marvel at. Time wore on, but it seemed to have little effect on Mr. Baggins. At ninety he was much the same as at fifty. At ninety-nine they began to call him well-preserved, but unchanged would have been nearer the mark. There were some that shook their heads and thought this was too muc

In [3]:
lotr_1_rtext.find('\xa0')

-1

In [4]:
import re

In [5]:
lotr_1_books = re.split('\r\nBook [IVX]+\r\n',lotr_1_rtext)[1:]
len(lotr_1_books)
[x[:100] for x in lotr_1_books]

['\r\n \r\nChapter 1\r\nA Long-expected Party\r\n\r\nWhen Mr. Bilbo Baggins of Bag End announced that he would s',
 '\r\n \r\nChapter 1\r\nMany Meetings\r\n\r\nFrodo woke and found himself lying in bed. At first he thought that']

In [6]:
lotr_1_book_chapters = [re.split('Chapter \d+\r\n.*\r\n', x.strip())[1:] for x in lotr_1_books]
[[ch[:100]+'...'+ch[-100:] for ch in bk[:2]+bk[-2:]] for bk in lotr_1_book_chapters]

[['\r\nWhen Mr. Bilbo Baggins of Bag End announced that he would shortly be celebrating his eleventy-firs...oaked figure quickly vanished into the twilight. Frodo did not see him again for a long time.\r\n\r\n \r\n',
  '\r\nThe talk did not die down in nine or even ninety-nine days. The second disappearance of Mr. Bilbo ...ted for a walk. ‘Me go and see Elves and all! Hooray!’ he shouted, and then burst into tears.\r\n\r\n \r\n',
  '\r\nAs they prepared for sleep in the inn at Bree, darkness lay on Buckland; a mist strayed in the del...dropping his sword, slipped the Ring from his finger and closed his right hand tight upon it.\r\n\r\n \r\n',
  '\r\nWhen Frodo came to himself he was still clutching the Ring desperately. He was lying by the fire, ...ing and confusion seemed to rise and engulf him together with his enemies. He heard and saw no more.'],
 ["\r\nFrodo woke and found himself lying in bed. At first he thought that he had slept late, after a lon...ght! I'll take a walk, I thin

In [7]:
lotr_1_book_ch_paras = [[re.split('\r\n\r\n\s*', ch.strip()) for ch in bk] for bk in lotr_1_book_chapters]
lotr_1_book_ch_paras[0][0][:4]

['When Mr. Bilbo Baggins of Bag End announced that he would shortly be celebrating his eleventy-first birthday with a party of special magnificence, there was much talk and excitement in Hobbiton.',
 'Bilbo was very rich and very peculiar, and had been the wonder of the Shire for sixty years, ever since his remarkable disappearance and unexpected return. The riches he had brought back from his travels had now become a local legend, and it was popularly believed, whatever the old folk might say, that the Hill at Bag End was full of tunnels stuffed with treasure. And if that was not enough for fame, there was also his prolonged vigour to marvel at. Time wore on, but it seemed to have little effect on Mr. Baggins. At ninety he was much the same as at fifty. At ninety-nine they began to call him well-preserved, but unchanged would have been nearer the mark. There were some that shook their heads and thought this was too much of a good thing; it seemed unfair that anyone should possess (app

In [8]:
pilcrow = '¶'
section = '§'
hyphen = '-' 
ndash = '\u2013' #'–'
mdash = '—'
elipses = '…'
print(hyphen, ndash, mdash)
hyphen == ndash

- – —


False

In [9]:
# May be good to concatenate the newlines,
# which seem to mess up the later tokenisation, as well as a few other
# things: ". . ." as elipses (utf8 …), and "--" as mdash (—)
import re
def sentence_prep(sentence, debug=False):
    sentence = ' '.join(re.split("\s+", sentence))
    sentence = re.sub('\s*\. \. \.', ' … ',sentence)
    sentence = re.sub('----', ' — ',sentence)
    sentence = re.sub('---', ' — ',sentence)
    sentence = re.sub('--', ' — ',sentence)
    sentence = re.sub('“', ' “ ',sentence)
    sentence = re.sub('”', ' ” ',sentence)
    sentence = re.sub('‘', ' ‘ ',sentence)
    sentence = re.sub('’([a-z])', '\'\\1', sentence) #eg "it isn’t natural"/"you’ve" -> "isn't"/"you've"
    sentence = re.sub('’', ' ’ ',sentence)
    return sentence

print(sentence_prep(
    'This “should” get replaced . . . and even this--one',
    debug=True,
))

This  “ should ”  get replaced …  and even this — one


In [10]:
#Split into sentences now
# Note full stop chars may not give the full sentences, eg
# "Mr. Bilbo Baggins"

#Import a premade sentence tokeniser
import nltk.data

#May need to download the data
nltk.download('punkt')
english_sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
#Option 2:
#from nltk.tokenize.punkt import PunktSentenceTokenizer
#punkt_tokenizer = PunktSentenceTokenizer(lotr_1_rtext, verbose=1)
#punkt_tokenizer

# Eg:
#punkt_tokenizer.sentences_from_text(lotr_1_rtext)[:20]

In [12]:
lotr_prep_bk_ch_para_sens = [[
    [
        [sentence_prep(sen, debug=False) for sen in english_sentence_tokenizer.tokenize(para)]
        for para in ch
    ] for ch in lotr_book
] for lotr_book in lotr_1_book_ch_paras]
#aoav_prep_paras = [
#    sentence_prep(para, debug=False)
#    for ch in aoav_ch_paras
#    for para in ch
#]
lotr_prep_bk_ch_para_sens[0][0][:4]

[['When Mr. Bilbo Baggins of Bag End announced that he would shortly be celebrating his eleventy-first birthday with a party of special magnificence, there was much talk and excitement in Hobbiton.'],
 ['Bilbo was very rich and very peculiar, and had been the wonder of the Shire for sixty years, ever since his remarkable disappearance and unexpected return.',
  'The riches he had brought back from his travels had now become a local legend, and it was popularly believed, whatever the old folk might say, that the Hill at Bag End was full of tunnels stuffed with treasure.',
  'And if that was not enough for fame, there was also his prolonged vigour to marvel at.',
  'Time wore on, but it seemed to have little effect on Mr. Baggins.',
  'At ninety he was much the same as at fifty.',
  'At ninety-nine they began to call him well-preserved, but unchanged would have been nearer the mark.',
  'There were some that shook their heads and thought this was too much of a good thing; it seemed unfai

In [13]:
print( '\n-----\n'.join(lotr_prep_bk_ch_para_sens[0][0][1][:]) )
print('===')
print( '\n-----\n'.join(lotr_prep_bk_ch_para_sens[1][6][0][:]) ) 

Bilbo was very rich and very peculiar, and had been the wonder of the Shire for sixty years, ever since his remarkable disappearance and unexpected return.
-----
The riches he had brought back from his travels had now become a local legend, and it was popularly believed, whatever the old folk might say, that the Hill at Bag End was full of tunnels stuffed with treasure.
-----
And if that was not enough for fame, there was also his prolonged vigour to marvel at.
-----
Time wore on, but it seemed to have little effect on Mr. Baggins.
-----
At ninety he was much the same as at fifty.
-----
At ninety-nine they began to call him well-preserved, but unchanged would have been nearer the mark.
-----
There were some that shook their heads and thought this was too much of a good thing; it seemed unfair that anyone should possess (apparently) perpetual youth as well as (reputedly) inexhaustible wealth.
===
The sun was sinking behind the mountains, and the shadows were deepening in the woods, when

In [14]:
lotr_all_stringable_sentences = []
for book in lotr_prep_bk_ch_para_sens:
    for ch in book:
        sen_list = []
        for para in ch:
            for sen in para:
                sen_list.append(sen)
        lotr_all_stringable_sentences.append(sen_list)
lotr_all_stringable_sentences[1]

['The talk did not die down in nine or even ninety-nine days.',
 'The second disappearance of Mr. Bilbo Baggins was discussed in Hobbiton, and indeed all over the Shire, for a year and a day, and was remembered much longer than that.',
 'It became a fireside-story for young hobbits; and eventually Mad Baggins, who used to vanish with a bang and a flash and reappear with bags of jewels and gold, became a favourite character of legend and lived on long after all the true events were forgotten.',
 'But in the meantime, the general opinion in the neighbourhood was that Bilbo, who had always been rather cracked, had at last gone quite mad, and had run off into the Blue.',
 'There he had undoubtedly fallen into a pool or a river and come to a tragic, but hardly an untimely, end.',
 'The blame was mostly laid on Gandalf.',
 " ‘ If only that dratted wizard will leave young Frodo alone, perhaps he'll settle down and grow some hobbit-sense, ’  they said.",
 'And to all appearance the wizard did 

In [15]:
#Start using tf for tokenization etc as well.
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token='<UNKNOWN>')

In [16]:
lotr_all_sentences_strung = []
for sentence_list in lotr_all_stringable_sentences:
    lotr_all_sentences_strung.extend(sentence_list)

lotr_all_sentences_strung[:10]

['When Mr. Bilbo Baggins of Bag End announced that he would shortly be celebrating his eleventy-first birthday with a party of special magnificence, there was much talk and excitement in Hobbiton.',
 'Bilbo was very rich and very peculiar, and had been the wonder of the Shire for sixty years, ever since his remarkable disappearance and unexpected return.',
 'The riches he had brought back from his travels had now become a local legend, and it was popularly believed, whatever the old folk might say, that the Hill at Bag End was full of tunnels stuffed with treasure.',
 'And if that was not enough for fame, there was also his prolonged vigour to marvel at.',
 'Time wore on, but it seemed to have little effect on Mr. Baggins.',
 'At ninety he was much the same as at fifty.',
 'At ninety-nine they began to call him well-preserved, but unchanged would have been nearer the mark.',
 'There were some that shook their heads and thought this was too much of a good thing; it seemed unfair that an

In [17]:
print("%d total sentences" % len(lotr_all_sentences_strung))
tokenizer.fit_on_texts(lotr_all_sentences_strung)
total_words = len(tokenizer.word_index) + 1 #word_index is a dict
print("%d total words" % total_words)

13392 total sentences
8661 total words


In [18]:
#Inspect how this processes:
test_sentence = lotr_all_sentences_strung[2] + ' - then an OOV word and mdash: ' + mdash
print(test_sentence)
tseq = tokenizer.texts_to_sequences([test_sentence])[0]
print(tseq)
print(tokenizer.sequences_to_texts([tseq]))

The riches he had brought back from his travels had now become a local legend, and it was popularly believed, whatever the old folk might say, that the Hill at Bag End was full of tunnels stuffed with treasure. - then an OOV word and mdash: —
[2, 5276, 7, 24, 353, 77, 39, 17, 5277, 24, 40, 494, 6, 2415, 1796, 3, 9, 12, 5278, 1265, 755, 2, 92, 200, 201, 135, 13, 2, 196, 25, 396, 131, 12, 387, 4, 2416, 3972, 27, 1194, 48, 90, 1, 418, 3, 1, 1628]
['the riches he had brought back from his travels had now become a local legend and it was popularly believed whatever the old folk might say that the hill at bag end was full of tunnels stuffed with treasure then an <UNKNOWN> word and <UNKNOWN> —']


In [19]:
# v2 - try to make sentences by adding the full stop punctuation
DOT_STR = "·" #One that wont get filtered by the default tokenisation
tokenizer.fit_on_texts([DOT_STR])

DOT_TOKEN = tokenizer.texts_to_sequences([DOT_STR])[0][0]
print(DOT_STR, "->", DOT_TOKEN)

total_words = len(tokenizer.word_index) + 1 #word_index is a dict
print("%d total words" % total_words)


· -> 8661
8662 total words


In [20]:
#ngram - note that paragraphs are stringable, but probably best to exclude chapters
## modified From the colab:
# create input sequences using list of tokens

DOT_SENTENCES = True #Whether to include a dot token between sentences

token_strings_list = []
for stringable_sentences in lotr_all_stringable_sentences:
    tokenized_sentences = tokenizer.texts_to_sequences(stringable_sentences)
    #print(stringable_sentences, tokenized_sentences)
    #break
    tseq = []
    for tlist in tokenized_sentences:
        tseq.extend(tlist)
        if DOT_SENTENCES:
            tseq.append(DOT_TOKEN)
    token_strings_list.append(tseq)

include_sentences_alone = True
if include_sentences_alone:
    for sentence_list in lotr_all_stringable_sentences:
        tokenized_sentences = tokenizer.texts_to_sequences(sentence_list)
        token_strings_list.extend(tokenized_sentences)


In [21]:
longest_sentence_len = max([(len(x),x) for x in token_strings_list])
print(
    len(token_strings_list), "sentence lists", ",",
    longest_sentence_len[0], "is longest len", " --> ",
    longest_sentence_len[1][:30],
    tokenizer.sequences_to_texts([longest_sentence_len[1][:30]])[0]
)


13414 sentence lists , 17647 is longest len  -->  [506, 164, 29, 988, 632, 596, 1496, 3, 105, 8661, 7, 375, 235, 2, 3820, 236, 2, 889, 1070, 1523, 3, 709, 2, 321, 781, 197, 1422, 236, 2, 81] next day frodo woke early feeling refreshed and well · he walked along the terraces above the loud flowing bruinen and watched the pale cool sun rise above the far


In [22]:
#Ought to limit the back check, maybe to 30 words
MAX_ALLOWED_SEQ_LENGTH = 30 #then there is 1 for the output

In [23]:
input_sequences=[]
for tseq in token_strings_list:
    for i in range(len(tseq)):
        #For each, collect the ngrams
        #And collect the [0:n] token 'n-grams'
        #  (feed in partial sentences as well)
        n_gram_sequence = tseq[max(0,i-MAX_ALLOWED_SEQ_LENGTH):i+1]
        input_sequences.append(n_gram_sequence)
    #print(tseq, n_gram_sequence, len(n_gram_sequence))
    #break

input_sequences[:4]

[[69], [69, 170], [69, 170, 112], [69, 170, 112, 374]]

In [24]:
#print(input_sentences[:2])
print(input_sequences[:20])
print(max(len(x) for x in input_sequences))

[[69], [69, 170], [69, 170, 112], [69, 170, 112, 374], [69, 170, 112, 374, 4], [69, 170, 112, 374, 4, 396], [69, 170, 112, 374, 4, 396, 131], [69, 170, 112, 374, 4, 396, 131, 3248], [69, 170, 112, 374, 4, 396, 131, 3248, 13], [69, 170, 112, 374, 4, 396, 131, 3248, 13, 7], [69, 170, 112, 374, 4, 396, 131, 3248, 13, 7, 59], [69, 170, 112, 374, 4, 396, 131, 3248, 13, 7, 59, 5273], [69, 170, 112, 374, 4, 396, 131, 3248, 13, 7, 59, 5273, 34], [69, 170, 112, 374, 4, 396, 131, 3248, 13, 7, 59, 5273, 34, 5274], [69, 170, 112, 374, 4, 396, 131, 3248, 13, 7, 59, 5273, 34, 5274, 17], [69, 170, 112, 374, 4, 396, 131, 3248, 13, 7, 59, 5273, 34, 5274, 17, 2411], [69, 170, 112, 374, 4, 396, 131, 3248, 13, 7, 59, 5273, 34, 5274, 17, 2411, 171], [69, 170, 112, 374, 4, 396, 131, 3248, 13, 7, 59, 5273, 34, 5274, 17, 2411, 171, 736], [69, 170, 112, 374, 4, 396, 131, 3248, 13, 7, 59, 5273, 34, 5274, 17, 2411, 171, 736, 27], [69, 170, 112, 374, 4, 396, 131, 3248, 13, 7, 59, 5273, 34, 5274, 17, 2411, 171, 73

In [25]:
#Prepare the sequences by padding them (left side zeros / "pre-padding")
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences



input_sequences_padded = np.array(
    pad_sequences(
        input_sequences,
        maxlen=MAX_ALLOWED_SEQ_LENGTH,
        padding='pre',
    )
)
input_sequences_padded

array([[   0,    0,    0, ...,    0,    0,   69],
       [   0,    0,    0, ...,    0,   69,  170],
       [   0,    0,    0, ...,   69,  170,  112],
       ...,
       [   0,    0,    0, ...,   55,    2,  156],
       [   0,    0,   48, ...,    2,  156,    4],
       [   0,   48, 8660, ...,  156,    4,  223]])

In [26]:
input_sequences_padded.shape

(380554, 30)

In [27]:
#This is enough for the prep.
# Now lets save, and then the data can be input into the processing script
#Need to save the tokeniser as well for encoding:
# then can use eg: tokenizer.texts_to_sequences([the_sentence])[0]

import pickle

with open('lotr_data_prep.pickle', 'wb') as handle:
    pickle.dump({
            'tokenizer': tokenizer,
            'train_tokens': input_sequences_padded,
        },
        handle,
    )